In [1]:
import uproot
import h5py
import numpy as np
import os
import pandas as pd
import awkward as awk
import glob
import heplot as hepl
import importlib
import sys
sys.path.append('..')

In [2]:
import src.selection as sele
import src.string_constants as stco
import src.generator as gene
import src.reader as read
import src.util as util

## read file

In [3]:
feature_names = ['el_e','mu_e','el_charge','mu_charge','el_pt','mu_pt', 'el_phi','mu_phi',\
                      'el_eta', 'mu_eta', 'jet_e', 'jet_pt', 'jet_GN2_pu', 'jet_GN2_pb', 'jet_GN2_pc', 'jet_truthflav',\
                'weight_mc', 'weight_pileup', 'weight_jvt', 'weight_leptonSF']

In [7]:
## read signal MC (one file per ee, mumu, tautau)

# read electron samples
path_ee = glob.glob(os.path.join(stco.in_dir_mc,'*'+stco.ds_ids_sig['ee'][0]+'*'))[0]
fname_ee = os.listdir(path_ee)[2]

In [5]:
tree = uproot.open(os.path.join(path_ee,fname_ee)+':nominal')
samples = tree.arrays(feature_names)

In [6]:
#samples = samples[:10]

In [7]:
samples

<Array [{el_e: [...], mu_e: [], ...}, ...] type='10 * {el_e: var * float32,...'>

## selection

In [8]:
# at least one jet
mask = awk.num(samples.jet_e) >= 1
samples = samples[mask]

In [9]:
samples

<Array [{el_e: [...], mu_e: [], ...}, ...] type='10 * {el_e: var * float32,...'>

In [28]:
jet_fields = [field for field in samples.fields if 'jet_' in field]
leading_jet_mask = awk.argmax(samples['jet_pt'],axis=1,keepdims=True)

In [29]:
samples.jet_pt

<Array [[7.35e+04, 2.69e+04], [...], ..., [5.69e+04]] type='10 * var * float32'>

In [30]:
leading_jet_mask

<Array [[0], [0], [0], [0], ..., [0], [0], [0], [0]] type='10 * 1 * ?int64'>

In [33]:
np.array(awk.flatten(samples['jet_pt'][leading_jet_mask]))

array([ 73523.664,  40686.355, 146718.3  ,  88198.17 ,  80824.   ,
        42598.355, 201598.77 ,  21264.797, 474529.38 ,  56892.95 ],
      dtype=float32)

In [34]:
samples['jet_pt_lead'] = np.array(awk.flatten(samples['jet_pt'][leading_jet_mask]))

In [35]:
samples

<Array [{el_e: [...], mu_e: [], ...}, ...] type='10 * {el_e: var * float32,...'>

In [37]:
for jet_field in jet_fields:
        samples[jet_field+'_lead'] = np.array(awk.flatten(samples[jet_field][leading_jet_mask])) # cant overwrite fields -> check

In [38]:
samples.jet_pt

<Array [[7.35e+04, 2.69e+04], [...], ..., [5.69e+04]] type='10 * var * float32'>

In [39]:
samples.jet_pt_lead

<Array [7.35e+04, 4.07e+04, ..., 4.75e+05, 5.69e+04] type='10 * float32'>

In [40]:
# z mass window [MeV]
z_m_min, z_m_max = 80e3, 100e3
# particle masses [MeV]
ele_m, mu_m = 511e-3, 105.7
# Z transverse momentum [MeV]
z_pt_min = 50e3

# compute invariant mass of electrons
ee_m = sele.calc_invariant_mass(samples.el_pt, samples.el_eta, samples.el_phi, part_m=ele_m)

# compute invariant mass of muons
mumu_m = sele.calc_invariant_mass(samples.mu_pt, samples.mu_eta, samples.mu_phi, part_m=mu_m)

# invariant mass of electrons 80-100 GeV
mask_ee_m = (ee_m > z_m_min) & (ee_m < z_m_max)

# invariant mass of muons 80-100 GeV
mask_mumu_m = (mumu_m > z_m_min) & (mumu_m < z_m_max)

# di-electron pt
mask_ee_pt = sele.calc_dilepton_pt(samples.el_pt, samples.el_eta, samples.el_phi) > z_pt_min

# di-muon pt
mask_mumu_pt = sele.calc_dilepton_pt(samples.mu_pt, samples.mu_eta, samples.mu_phi) > z_pt_min

# exactly two electrons of opposite charge
mask_ee = (awk.num(samples.el_e) == 2) & (awk.sum(samples.el_charge,axis=1) == 0) 

# exactly two muons of opposite charge
mask_mumu = (awk.num(samples.mu_e) == 2) & (awk.sum(samples.mu_charge,axis=1) == 0)

# exclude events with four leptons
mask_4l = mask_ee & mask_mumu
mask_ee = mask_ee & ~mask_4l
mask_mumu = mask_mumu & ~mask_4l

# electrons in Z invariant mass and transverse momentum window
mask_ee = mask_ee & mask_ee_m & mask_ee_pt

# muons in Z invariant mass and transverse momentum window
mask_mumu = mask_mumu & mask_mumu_m & mask_mumu_pt

# 2 electrons or 2 muons with Z invariant mass
mask = mask_ee | mask_mumu

In [41]:
mask

<Array [True, True, True, True, ..., True, False, True, False] type='10 * bool'>

In [42]:
samples[mask]

<Array [{el_e: [...], mu_e: [], ...}, ...] type='7 * {el_e: var * float32, ...'>

In [43]:
sele_samp = samples[mask]

In [44]:
jetU, jetUNot = util.split_light_vs_nonlight_jet(sele_samp)

In [45]:
jetU

<Array [{el_e: [...], mu_e: [], ...}, ...] type='3 * {el_e: var * float32, ...'>

In [46]:
jpt = np.array(jetU.jet_pt_lead)

In [50]:
inputs_jetU = pd.DataFrame({'pt':jpt,'id':0})

In [51]:
inputs_jetUNot = pd.DataFrame({'pt':np.array(jetUNot.jet_pt_lead),'id':1})

In [52]:
inputs_jetU

,pt,id
0,40686.355469,0
1,146718.296875,0
2,201598.765625,0


In [53]:
inputs_jetUNot

,pt,id
0,73523.664062,1
1,88198.171875,1
2,80824.000000,1
3,474529.375000,1


In [54]:
sele_samp.jet_pt_lead

<Array [7.35e+04, 4.07e+04, ..., 2.02e+05, 4.75e+05] type='7 * float32'>

In [55]:
sele_samp.jet_truthflav_lead

<Array [5, 0, 0, 5, 5, 0, 5] type='7 * int32'>

In [59]:
(pd.concat([inputs_jetU,inputs_jetUNot],ignore_index=True)).sample(frac=1.)

,pt,id
1,146718.296875,0
6,474529.375000,1
2,201598.765625,0
5,80824.000000,1
4,88198.171875,1
0,40686.355469,0
3,73523.664062,1


In [57]:
def make_light_vs_nonlight_input_dataset(samples:awk.highlevel.Array) -> pd.DataFrame:

    eps = 1e-2
    
    # split samples into light and non-light jet classes
    jetU, jetUNot = util.split_light_vs_nonlight_jet(samples)

    # get relevant features
    in_jetU = pd.DataFrame({'pt':np.array(jetU.jet_pt_lead),'id':0})
    in_jetUNot = pd.DataFrame({'pt':np.array(jetUNot.jet_pt_lead),'id':1})

    # normalize
    u_min, u_max, u_mean, un_min, un_max, un_mean = in_jetU.pt.min()-eps, in_jetU.pt.max(), in_jetU.pt.mean(), in_jetUNot.pt.min()-eps, in_jetUNot.pt.max(), in_jetUNot.pt.mean()
    in_jetU.pt = (in_jetU.pt - u_min) / (u_max - u_min)
    in_jetUNot.pt = (in_jetUNot.pt - un_min) / (un_max - un_min)

    # concatenate and shuffles
    inputs = (pd.concat([in_jetU,in_jetUNot])).sample(frac=1.)

    return inputs


In [85]:
inputs = make_light_vs_nonlight_input_dataset(sele_samp)

In [86]:
inputs

,pt,id
0,7.282688e-08,0
2,1.000000e+00,0
3,1.000000e+00,1
1,3.659428e-02,1
2,1.820509e-02,1
1,6.589420e-01,0
0,1.948227e-08,1


## leading jet selections 

In [44]:
feature_names_in = ['el_e','mu_e','el_charge','mu_charge','el_pt','mu_pt', 'el_phi','mu_phi',\
                    'el_eta', 'mu_eta', 'jet_e', 'jet_pt', 'jet_truthflav',\
                	'weight_mc', 'weight_pileup', 'weight_jvt', 'weight_leptonSF']
feature_names_out = ['jet_pt_lead','jet_truthflav_lead',\
                	'weight_mc', 'weight_pileup', 'weight_jvt', 'weight_leptonSF']

In [45]:
filename = '/eos/atlas/atlasgroupdisk/perf-flavtag/dq2/rucio/user/ltoffoli/bf/13/user.ltoffoli.37305913._000001.output.root'

In [46]:
tree = uproot.open(filename+':nominal')
samples = tree.arrays(feature_names_in)

In [47]:
samples

<Array [{el_e: [...], mu_e: [], ...}, ...] type='2205222 * {el_e: var * flo...'>

In [48]:
mask = awk.num(samples.jet_e) >= 1

In [49]:
mask

<Array [True, True, True, True, ..., True, True, False] type='2205222 * bool'>

In [50]:
np.sum(mask)

2047206

In [51]:
samples = samples[mask] # only events with at least one jet

In [52]:
samples

<Array [{el_e: [...], mu_e: [], ...}, ...] type='2047206 * {el_e: var * flo...'>

In [53]:
jet_fields = [field for field in samples.fields if 'jet_' in field]

In [63]:
leading_jet_mask = awk.argmax(samples['jet_pt'],axis=1,keepdims=True,mask_identity=False)

In [64]:
leading_jet_mask

<Array [[0], [0], [0], [0], ..., [0], [0], [0], [0]] type='2047206 * 1 * int64'>

In [65]:
min(awk.num(samples['jet_pt']))

1

In [66]:
max(awk.num(samples['jet_pt']))

16

In [67]:
def add_leading_pt_jet_features(samples:awk.highlevel.Array) -> awk.highlevel.Array:

    for jet_field in jet_fields:
        samples[jet_field+'_lead'] = awk.flatten(samples[jet_field][leading_jet_mask]) # cant overwrite fields -> check

    return samples


In [68]:
samples = add_leading_pt_jet_features(samples)

In [69]:
samples

<Array [{el_e: [...], mu_e: [], ...}, ...] type='2047206 * {el_e: var * flo...'>

In [70]:
import ntuple_root_to_h5_converter.src.util as cnv_util

In [71]:
sin = samples[['jet_pt_lead','jet_truthflav_lead']]

In [72]:
sin

<Array [{jet_pt_lead: [...], ...}, ..., {...}] type='2047206 * {jet_pt_lead...'>

In [73]:
sin.jet_truthflav_lead

<Array [[0, 0, 5], [0, 0, 5], ..., [0, 0, 5]] type='2047206 * var * int32'>